<a href="https://colab.research.google.com/drive/1l303DcxP66J8i_mNxSKRPirU2HXXq5OB?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Search & RAG with Cohere & Pinecone

In [1]:
! pip install cohere annoy pinecone-client -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [2]:
import cohere
import os
import random
import pandas as pd
from annoy import AnnoyIndex
import numpy as np

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [3]:
api_key = "" # add your cohere API key here
co = cohere.Client(api_key)

### Import a list of documents

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [56]:
raw_df = pd.read_csv("/content/gdrive/MyDrive/data.csv", index_col=0)
print(raw_df.shape)
hq_df = raw_df
hq_df.head()

(1191, 5)


,Abstract,Authors,Publication Year,url,Summary
Title,,,,,
Change in Estradiol Levels among Premenopausal Patients with Breast Cancer Treated Using Leuprolide Acetate 11.25 Milligrams 3-Month Depot and Tamoxifen.,The combination of luteinizing hormone-releasi...,['Lee YJ; Wu ZY; Kim HJ; Lee JW; Chung IY; Kim...,2020,https://pubmed.ncbi.nlm.nih.gov/33154830,318 patients were included in the study and an...
Age at menarche and premenstrual syndrome in adolescent girls with intellectual disability in Bosnia and Herzegovina.,The issues involving menstruation are the topi...,['Ibralic I; Sinanovic O; Memisevic H'],2010,https://pubmed.ncbi.nlm.nih.gov/20207519,the study was conducted in a study of 31 adole...
Archaeological evidence of an ethnographically documented Australian Aboriginal ritual dated to the last ice age.,"In societies without writing, ethnographically...",['David B; Mullett R; Wright N; Stephenson B; ...,2024,https://pubmed.ncbi.nlm.nih.gov/38951612,ethnographicly documented rituals date back to...
"Impact of the 2008 <math xmlns=""http://www.w3.org/1998/Math/MathML""><msub><mi>M</mi> <mi>W</mi></msub> </math> 7.9 Great Wenchuan earthquake on South China microplate motion.",Tectonic plate motions drive the earthquake cy...,['Iaffaldano G; Martin de Blas J; Rui X; Stamp...,2024,https://pubmed.ncbi.nlm.nih.gov/39014128,tectonic plate motions drive the earthquake cy...
Characteristics of the menstrual cycle in 13-year-old Flemish girls and the impact of menstrual symptoms on social life.,This study aimed to investigate the characteri...,"[""Hoppenbrouwers K; Roelants M; Meuleman C; Ri...",2016,https://pubmed.ncbi.nlm.nih.gov/26670027,study aims to investigate the characteristics ...


In [58]:
df = hq_df.drop(columns=['Summary'])

In [59]:
df.shape

(1191, 4)

In [60]:
df.head()

,Abstract,Authors,Publication Year,url
Title,,,,
Change in Estradiol Levels among Premenopausal Patients with Breast Cancer Treated Using Leuprolide Acetate 11.25 Milligrams 3-Month Depot and Tamoxifen.,The combination of luteinizing hormone-releasi...,['Lee YJ; Wu ZY; Kim HJ; Lee JW; Chung IY; Kim...,2020,https://pubmed.ncbi.nlm.nih.gov/33154830
Age at menarche and premenstrual syndrome in adolescent girls with intellectual disability in Bosnia and Herzegovina.,The issues involving menstruation are the topi...,['Ibralic I; Sinanovic O; Memisevic H'],2010,https://pubmed.ncbi.nlm.nih.gov/20207519
Archaeological evidence of an ethnographically documented Australian Aboriginal ritual dated to the last ice age.,"In societies without writing, ethnographically...",['David B; Mullett R; Wright N; Stephenson B; ...,2024,https://pubmed.ncbi.nlm.nih.gov/38951612
"Impact of the 2008 <math xmlns=""http://www.w3.org/1998/Math/MathML""><msub><mi>M</mi> <mi>W</mi></msub> </math> 7.9 Great Wenchuan earthquake on South China microplate motion.",Tectonic plate motions drive the earthquake cy...,['Iaffaldano G; Martin de Blas J; Rui X; Stamp...,2024,https://pubmed.ncbi.nlm.nih.gov/39014128
Characteristics of the menstrual cycle in 13-year-old Flemish girls and the impact of menstrual symptoms on social life.,This study aimed to investigate the characteri...,"[""Hoppenbrouwers K; Roelants M; Meuleman C; Ri...",2016,https://pubmed.ncbi.nlm.nih.gov/26670027


In [61]:
df.isna()

,Abstract,Authors,Publication Year,url
Title,,,,
Change in Estradiol Levels among Premenopausal Patients with Breast Cancer Treated Using Leuprolide Acetate 11.25 Milligrams 3-Month Depot and Tamoxifen.,False,False,False,False
Age at menarche and premenstrual syndrome in adolescent girls with intellectual disability in Bosnia and Herzegovina.,False,False,False,False
Archaeological evidence of an ethnographically documented Australian Aboriginal ritual dated to the last ice age.,False,False,False,False
"Impact of the 2008 <math xmlns=""http://www.w3.org/1998/Math/MathML""><msub><mi>M</mi> <mi>W</mi></msub> </math> 7.9 Great Wenchuan earthquake on South China microplate motion.",False,False,False,False
Characteristics of the menstrual cycle in 13-year-old Flemish girls and the impact of menstrual symptoms on social life.,False,False,False,False
...,...,...,...,...
Unusual Manifestation of Membranous Dysmenorrhea: Case Report.,False,False,False,False
Efficacy and Pain Tolerance of Alexandrite Laser Hair Removal at Different Stages of the Menstrual Cycle.,False,False,False,False
Assessment of urinary cotinine levels in women with gynecological complaints at a tertiary care hospital: A pilot study.,False,False,False,False


In [62]:
df.isna().sum()

,0
Abstract,24
Authors,1
Publication Year,0
url,0


In [63]:
df.dropna(inplace=True)

In [64]:
df.head()

,Abstract,Authors,Publication Year,url
Title,,,,
Change in Estradiol Levels among Premenopausal Patients with Breast Cancer Treated Using Leuprolide Acetate 11.25 Milligrams 3-Month Depot and Tamoxifen.,The combination of luteinizing hormone-releasi...,['Lee YJ; Wu ZY; Kim HJ; Lee JW; Chung IY; Kim...,2020,https://pubmed.ncbi.nlm.nih.gov/33154830
Age at menarche and premenstrual syndrome in adolescent girls with intellectual disability in Bosnia and Herzegovina.,The issues involving menstruation are the topi...,['Ibralic I; Sinanovic O; Memisevic H'],2010,https://pubmed.ncbi.nlm.nih.gov/20207519
Archaeological evidence of an ethnographically documented Australian Aboriginal ritual dated to the last ice age.,"In societies without writing, ethnographically...",['David B; Mullett R; Wright N; Stephenson B; ...,2024,https://pubmed.ncbi.nlm.nih.gov/38951612
"Impact of the 2008 <math xmlns=""http://www.w3.org/1998/Math/MathML""><msub><mi>M</mi> <mi>W</mi></msub> </math> 7.9 Great Wenchuan earthquake on South China microplate motion.",Tectonic plate motions drive the earthquake cy...,['Iaffaldano G; Martin de Blas J; Rui X; Stamp...,2024,https://pubmed.ncbi.nlm.nih.gov/39014128
Characteristics of the menstrual cycle in 13-year-old Flemish girls and the impact of menstrual symptoms on social life.,This study aimed to investigate the characteri...,"[""Hoppenbrouwers K; Roelants M; Meuleman C; Ri...",2016,https://pubmed.ncbi.nlm.nih.gov/26670027


In [65]:
df.shape

(1167, 4)

In [67]:
# Print a few sample documents
texts = df["Abstract"].tolist()

random.seed(100)
for item in random.sample(texts, 5):
  print(item)

Migraine and epilepsy are two episodic disorders that share common pathophysiological mechanisms. The aim of our research was to assess the possible shared etiopathogenesis by analyzing the relations of headache, and seizure triggers, based on information obtained from a national cohort surveying the headache characteristics of 809 patients who had been diagnosed with idiopathic/genetic epilepsy. Our study utilized data from a multi-center, nationwide investigation of headaches in 809 patients with idiopathic/genetic epilepsy. Out of these, 508 patients reported complaints related to any type of headache (333 Migraines, 175 Headaches of other types). In the initial phase of the study encompassing the entire sample of 809 epilepsy patients, differences in seizure triggers were assessed between the migraine group (n = 333) and the non-migraine group (n = 476). Additionally, the subsequent part of the study pertains to a subgroup of the entire patient group, namely those affected by all t

### Embed the documents and store them in an index

In [68]:
response = co.embed(texts=texts, model='multilingual-22-12').embeddings
embeds = np.array(response)
# Embed the documents and store in index
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(100) # 10 trees
search_index.save('data_index.ann')

True

In [69]:
embeds.shape

(1167, 768)

### Enter a query

In [70]:
# They are all the same ingredients
# Expected: Output recipes are the same
queries = ["What is the best menstrual cycle?"]

queries_lang = ["English"]

### Return the results most similar to the query



In [73]:
results_list = []

for idx, q in enumerate(queries):

    # Retrieve the nearest neighbors
    query_embed = co.embed(texts=[q], model='multilingual-22-12').embeddings
    similar_item_ids, _ = search_index.get_nns_by_vector(query_embed[0], 5, include_distances=True)

    if len(similar_item_ids) >= 2:
        # Format the results
        results = pd.DataFrame(data={'Abstract': df.iloc[similar_item_ids[0]]['Abstract'],
                                     'Authors': df.iloc[similar_item_ids[0]]['Authors'],
                                     'Publication Year': df.iloc[similar_item_ids[0]]['Publication Year'],}, index=[idx])

        results_list.append(results)
        print(f"Query:'{q}'\nNearest neighbors:")
        print(queries_lang[idx])
        print(results)
        print("\n")
    else:
        print("Not enough similar items found for query:", q)

Query:'What is the best menstrual cycle?'
Nearest neighbors:
English
                                            Abstract  \
0  Women are considered to have an irregular mens...   

                                 Authors Publication Year  
0  ['Attia GM; Alharbi OA; Aljohani RM']             2023  




In [77]:
dimension = embeds.shape[1]
dimension

768

### Initialise the pinecone index


In [86]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="") # add your pinecone API key here

index_name = 'mh'

# if the index does not exist, we create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
    name=index_name,
    dimension=embeds.shape[1], # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

# connect to index
index = pc.Index(index_name)

In [95]:
# Assuming the dimension is 768
dimension = 768

# If the index already exists, you may need to delete it first
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# Create the index with the correct dimension
pc.create_index(
    index_name,
    dimension=dimension,
    metric='cosine',
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

# Connect to the index
index = pc.Index(index_name)


In [96]:
import numpy as np

shape = np.array(embeds).shape
shape

(1167, 768)

### Upsert the data into pinecone


In [97]:
batch_size = 128

# Assuming shape[0] refers to the number of rows in your DataFrame
ids = [str(i) for i in range(shape[0])]

# Create list of metadata dictionaries
meta = [
    {'Abstract': abstract, 'Authors': authors, 'Publication Year': publication_year}
    for abstract, authors, publication_year in zip(df['Abstract'], df['Authors'], df['Publication Year'])
]

# Create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, embeds, meta))

# Upsert in batches
for i in range(0, shape[0], batch_size):
    i_end = min(i + batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

# Let's view the index statistics
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1024}},
 'total_vector_count': 1024}

### Return the results most similar to the query from pinecone


In [98]:
query = "What is the best menstrual cycle?"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='multilingual-22-12',
    truncate='NONE'
).embeddings

print(np.array(xq).shape)

# query, returning the top 10 most similar results
res = index.query(vector=xq, top_k=10, include_metadata=True)
res

(1, 768)


{'matches': [{'id': '636',
              'metadata': {'Abstract': 'Women are considered to have an '
                                       'irregular menstrual cycle if their '
                                       'cycle length is less than 21 days or '
                                       'more than 35 days, accompanied by less '
                                       'or very severe blood flow. The '
                                       'prevalence of menstrual cycle '
                                       'irregularities varies across '
                                       'countries. Irregular periods can occur '
                                       "due to changes in the body's levels of "
                                       'estrogen and progesterone hormones, '
                                       'which disrupt the normal pattern of '
                                       'the period. Menstrual irregularity has '
                                       'been fo

In [99]:
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['Abstract']}")
    print(f"{match['score']:.2f}: {match['metadata']['Authors']}")
    print(f"{match['score']:.2f}: {match['metadata']['Publication Year']}")

0.92: Women are considered to have an irregular menstrual cycle if their cycle length is less than 21 days or more than 35 days, accompanied by less or very severe blood flow. The prevalence of menstrual cycle irregularities varies across countries. Irregular periods can occur due to changes in the body's levels of estrogen and progesterone hormones, which disrupt the normal pattern of the period. Menstrual irregularity has been found to be associated with various diseases and medical conditions, such as metabolic syndrome, coronary heart disease, type 2 diabetes mellitus, and rheumatoid arthritis. Anemia, osteoporosis, psychological problems, impaired quality of life, and infertility have also been recorded. Moreover, a significant correlation between irregular periods and the risk of developing pregnancy-related hypertensive disorders, as well as an increased risk of adverse obstetric and neonatal outcomes, has been proven. Therefore, irregular menstruation is considered an important